1. Загружаем набор данных о диабете и разделяем его на обучающую и тестовую выборки.
2. Определяем функцию `objective`, которая принимает `trial` и настраивает гиперпараметр `alpha` для Ridge-регрессии. Мы используем `suggest_loguniform`, чтобы задать логарифмическое пространство для `alpha`.
3. Обучаем модель и вычисляем MSE, который будем минимизировать.
4. Создаем объект `study`, чтобы оптимизировать гиперпараметры, и запускаем процесс оптимизации с заданным количеством операций (в данном случае 100).
5. После нахождения лучших гиперпараметров, мы обучаем финальную модель и рассчитываем R² и MSE на тестовых данных.
6. В конце выводим лучшие гиперпараметры, а также значения R² и MSE.

In [1]:
import numpy as np
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

c:\Users\motyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загружаем данные
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

In [3]:
# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Функция для оптимизации гиперпараметров
def objective(trial):
    # Определяем гиперпараметры
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)
    model = Ridge(alpha=alpha)

    # Обучаем модель
    model.fit(X_train, y_train)

    # Предсказываем
    y_pred = model.predict(X_test)

    # Вычисляем ошибки
    mse = mean_squared_error(y_test, y_pred)
    return mse  # Возвращаем MSE для минимизации

In [5]:
# Настраиваем оптимизацию гиперпараметров
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Выводим лучшие гиперпараметры
best_params = study.best_params
best_model = Ridge(alpha=best_params['alpha'])
best_model.fit(X_train, y_train)

# Делаем предсказания на тестовом наборе
y_pred = best_model.predict(X_test)

[I 2025-03-08 11:34:24,315] A new study created in memory with name: no-name-8a02bd59-97c9-4b43-bd5b-c5b8c8e8d61d
C:\Users\motyn\AppData\Local\Temp\ipykernel_18496\645464418.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)
[I 2025-03-08 11:34:24,333] Trial 0 finished with value: 3055.5571973528067 and parameters: {'alpha': 0.9337586009580997}. Best is trial 0 with value: 3055.5571973528067.
C:\Users\motyn\AppData\Local\Temp\ipykernel_18496\645464418.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)
[I 2025-03-08 11:34:24,339] Trial 1 finished with value: 2900.0

In [6]:
# Вычисляем R² и MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Лучшие гиперпараметры: alpha = {best_params['alpha']}")
print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")

Лучшие гиперпараметры: alpha = 0.15461367300984613
R²: 0.4615
MSE: 2853.2743
